# LangGraph Practice

Welcome to LangGraph practice notebook. This is where i'll build and test graphs.

In [ ]:
! pip install dotenv

In [ ]:
import os
from dotenv import load_dotenv
# Load environment variables
load_dotenv()

print("Environment loaded.")

Importing Required Pacakages


In [ ]:
! pip install langchain
! pip install langgraph 
! pip istall langchain_core
! pip install langchain_google_generativeai
! pip install mypy-ipython


In [ ]:
import langgraph.graph

# List all components of the langgraph.graph module
print(dir(langgraph.graph))

In [ ]:
%load_ext mypy_ipython


In [ ]:
from langgraph.graph import START, END, StateGraph
from typing_extensions import TypedDict 
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.runnables import Runnable
from _collections_abc import Sequence
from typing_extensions import TypedDict
from langchain_core.messages import AIMessage, HumanMessage, BaseMessage
from typing import Literal


Define State

In [ ]:
class State(TypedDict):
    messages: Sequence[BaseMessage]

Define Node

In [ ]:
chat = ChatGoogleGenerativeAI(model="gemini-flash-latest", temperature=0, seed  =365, max_tokens=100)


In [ ]:
def ask_question(state:State)->State:
    print("entered ask_question")
    user_input=input("what is your question?")
    return State(messages=[HumanMessage(content=user_input)])


In [ ]:
ask_question(State(messages=[]))

In [ ]:
def chatbot(state: State) -> State:
    print("entered chatbot node")
    response = chat.invoke(state["messages"])
    response.pretty_print()
    return State(messages=response)

In [ ]:
def ask_another_question(state:State)->State:
    print("entered ask_another_question")
    user_input=input("do you wanna ask another question? (yes/no)")
    return State(messages=[HumanMessage(content=user_input)])

test_state2=State(messages=[])
ask_another_question(test_state2)

Define routing function

In [ ]:
def route(state:State)->Literal["ask_question", "__end__"]:
    
    if state["messages"][0].content == "yes":
        return "ask_question"
    else:
        return "__end__"

Define the graph 

In [ ]:
graph = StateGraph(State)

In [ ]:
graph.add_node("ask_question", ask_question)
graph.add_node("chatbot", chatbot)
graph.add_node("ask_another_question", ask_another_question)

graph.add_edge(START, "ask_question")
graph.add_edge("ask_question", "chatbot")
graph.add_edge("chatbot", "ask_another_question")
graph.add_conditional_edges(source = "ask_another_question", 
                            path = route)

In [ ]:
graph_compiled = graph.compile()

In [ ]:
graph_compiled

Testing the graph

In [ ]:
graph_compiled.invoke(State(messages = []))